In [1]:
import pandas as pd
import numpy as np

# Introduction

After working on the problem for a bit, the main challenge was to figure out the patter of how numbers are adjacent to each other. I then realized there are two types of tiles: "corner" tiles and "interior" tiles, for each layer. Every tile has two adjacent neighbors on the same level, but the corner vs. interior tiles have different behavior with respect to neighbors on the previous and next layers.

Corner tiles have $1$ neighbor on the previous layer and $3$ neighbors on the next layer (e.g., $8$ or $10$ or $12$ in the diagram on ProjectEuler). Interior tiles have $2$ neighbors on the previous layer and $2$ neighbors on the next layer (e.g., $9$ or $11$ or $13$ on the diagram on ProjectEuler). Also, there are nice patterns of indexing for each layer--each tile within a layer sort of picks up where the previous one left off and continues from there. This all results in nice behavior but it's not scalable since we have to produce every tile.

# Patterns

Like some of the recent problems, we look for patterns to reduce the search space. What we see by looking at the first $100$ layers is that actually every solution is either the first or last tile in a given layer. So that makes our life a lot easier--every layer we just have to produce the first and last tile, if this pattern holds. To show the pattern holds, we can do some math. Let's ignore the first layer, for now, since that one has slighlty different rules. So start on the layer beginning with the $8$ tile. Note: by starting from the second layer, we know that any difference between layer $l$ and $l-1$ is going to be $>2$, for any $l \geq 2$--this means if we are looking "between" layers, we're looking for odd primes.

# Proof of Reduction of Search Space

First rule out interiors. Suppose we have an interior tile $n$ that is not the last tile in a layer. We can treat the last tile separately (even though it's interior) since it is not a neighbor with $n+1$. Then in the current layer it has two adjacent tiles, $n+1$ and $n-1$, both of which have difference of $1$ which is not prime. So this leaves the previous and next layers. In the previous layer, $n$ is adjacent to $2$ adjacent tiles: call these $x$ and $x+1$. If we know $|n-x|$ is prime (and therefore odd), then $|n-(x+1)|$ must be even, and therefore not prime (since we know $|n-x| > 2$). This means we can have at most one prime from the previous layer. In the next layer, $n$ is adjacent to $2$ tiles: call these $y$ and $y+1$. Again, if we know $|n-y|$ is prime, then we know $|n- (y+1)|$ cannot be prime. This means for interior tiles, $PD(n) \leq 2$. So we can ignore them altogether.

Next let's look at corner tiles, of which there are $6$ per layer. Suppose $n$ is corner tile $c$ (counting counter-clockwise, where $0$ is the start tile) on layer $l$. We can treat the start tile separately since its $n-1$ is not its neighbor, so assume $1 \leq c \leq 5$. As in the interior case, the adjacent tiles are $n+1$ and $n-1$ which have differences of $1$, so those are not prime. On the next layer, we know $n$ is adjacent to $3$ tiles: call these $y-1, y, y+1$. Note that $y-1$ and $y+1$ are the same parity and that $y$ is a corner tile on the next layer. On the previous layer, $n$ is adjacent to $1$ tile: call this $x$. Note $x$ is also a corner tile. If $c$ is even, then $x, y$ and $n$ have the same parity, so $|n-y|$ and $|n-x|$ are even (not prime) which implies $|n-(y+1)|$ and $|n-(y-1)|$ are both odd, so there are only 2 possible primes. If $c$ is odd, then $x, y$ are both the same parity and $n$ has the opposite. Suppose $x,y$ are odd and $n$ is even. Then $|n-x|$ and $|n-y|$ are both odd which implies $|n-(y+1)|$ and $|n-(y-1)|$ are even (not prime), which again means only 2 possible primes. So for these tiles, $PD(n) \leq 2$.

# Implementation

This leaves only the first and last tile in each layer to check, except for the layers $0$ and $1$, where we should just check all tiles. For the remaining layers, we just check the first and last tile. In particular, we need the difference between the first and last tile to be prime in a layer for $PD(n) = 3$.

We initialize the first tile $a_1 = 2$ and the last tile $b_1 = 7$.(the first layer). On layer $l$ the first tile is $a_l = b_{l-1} + 1$ and the second is $b_{l} = b_{l-1} + 6l$. The numbers to check for the first tile $a_l$ would be 
1. $a_{l+1} - a_{l} + 1 = 6l + 1$, 
2. $b_{l} - a_{l} = 6l - 1$, 
3. $(b_{l} + 6(l+1)) - a_{l} = 12l + 5$

The numbers to check for the last tile $b_{l}$ would be 
1. $b_{l+1} - b_{l} - 1 = 6(l+1) - 1 = 6l + 5$
2. $b_{l} - a_{l} = 6l - 1$
3. $b_{l} - a_{l-1} = b_{l-1} + 6l - b_{l-2} - 1 = b_{l-2} + 6(l-1) + 6l - b_{l-2} - 1 = 12l - 7$

So ultimately, we can just get the values by layer number, after checking tiles $1$ to $7$ manually. It's easy to just brute-force that $1$ and $2$ are the only of the first $7$ tiles that satisfy $PD(n) = 3$. The final algorithm is elegant and instant.

In [99]:
def sieve(n):
    arr = [0,0,1] + [1,0]*(n//2 + 1)
    i = 3
    while i*i <= n:
        if arr[i]:  
            arr[i*i::2*i] = [0]*len(arr[i*i::2*i])
        i += 2

    ret = []
    for (i, p) in enumerate(arr):
        if p:
            ret.append(i)

    return arr, ret

pbs, ps = sieve(10**7)

### Producing every tile in the hex, first 100 layers.

In [100]:
# numbers by layer of splits up each layer and gives the values in each layer
# For a particular tile:
#       [..., [..., (val, prev, next), ...], ...]
# val is the value of the tile
# prev tells you adjacent tiles on previous layer by index: [0] means look at 0th index from previous layer
# next tells you adjacent tiles on next layer by index: [0] means look at 0th index on next layer
# -- note: len(prev) + len(next) = 4, since there are always two neighbors on same layer as tile
numbers_by_layer = [
    [(1,[],[0,1,2,3,4,5])], 
    [(2,[0],[-1,0,1]),(3,[0],[1,2,3]),(4,[0],[3,4,5]),(5,[0],[5,6,7]),(6,[0],[7,8,9]),(7,[0],[9,10,11])]
]

for i in range(3,100):
    layer_num = len(numbers_by_layer)
    layer_len = len(numbers_by_layer[-1]) + 6
    layer_start = numbers_by_layer[-1][-1][0] + 1

    # new_layer[0] always is top node--has same index on previous layer and [-1,0,1] on next layer
    new_layer = [(layer_start, [0], [-1,0,1])]
    for j in range(1,layer_len):
        prev_max = new_layer[j-1][1][-1]
        next_max = new_layer[j-1][2][-1]
        if j % (i-1) == 0:
            new_layer.append((layer_start + j, [prev_max], [next_max, next_max+1, next_max+2]))
        else:
            new_layer.append((layer_start + j, [prev_max, prev_max+1], [next_max, next_max+1]))

    numbers_by_layer.append(new_layer)

max_num = numbers_by_layer[-1][-1][0]

In [102]:
tiles = {1: [2,3,4,5,6,7]}
print(1, ":", (tiles[1], 3))
overall_cntr = 1
max_diff = 5

for layer in range(1,len(numbers_by_layer)-1):
    prev_layer = numbers_by_layer[layer-1]
    curr_layer = numbers_by_layer[layer]
    next_layer = numbers_by_layer[layer+1]
    for i, (val, prev, next) in enumerate(curr_layer):
        cntr = 0
        neighbors = [curr_layer[i-1][0], curr_layer[i+1 if i+1 < len(curr_layer) else 0][0]]
        for p in prev:
            neighbors.append(prev_layer[p if p < len(prev_layer) else 0][0])
        for n in next:
            neighbors.append(next_layer[n if n < len(next_layer) else 0][0])

        for x in neighbors:
            max_diff = max(abs(val - x), max_diff)
            if pbs[abs(val - x)] == 1:
                cntr += 1
        
        if cntr == 3:
            print(val, ":", (neighbors, cntr), 'here' if (i == 0 or i == len(curr_layer) - 1) else '')
            overall_cntr += 1
        tiles[val] = (neighbors.copy(), cntr)

print(overall_cntr)

1 : ([2, 3, 4, 5, 6, 7], 3)
2 : ([7, 3, 1, 19, 8, 9], 3) here
8 : ([19, 9, 2, 37, 20, 21], 3) here
19 : ([18, 8, 7, 2, 36, 37], 3) here
20 : ([37, 21, 8, 61, 38, 39], 3) here
37 : ([36, 20, 19, 8, 60, 61], 3) here
61 : ([60, 38, 37, 20, 90, 91], 3) here
128 : ([169, 129, 92, 217, 170, 171], 3) here
217 : ([216, 170, 169, 128, 270, 271], 3) here
271 : ([270, 218, 217, 170, 330, 331], 3) here
398 : ([469, 399, 332, 547, 470, 471], 3) here
919 : ([918, 818, 817, 722, 1026, 1027], 3) here
1519 : ([1518, 1388, 1387, 1262, 1656, 1657], 3) here
1520 : ([1657, 1521, 1388, 1801, 1658, 1659], 3) here
2978 : ([3169, 2979, 2792, 3367, 3170, 3171], 3) here
3170 : ([3367, 3171, 2978, 3571, 3368, 3369], 3) here
4220 : ([4447, 4221, 3998, 4681, 4448, 4449], 3) here
4447 : ([4446, 4220, 4219, 3998, 4680, 4681], 3) here
4681 : ([4680, 4448, 4447, 4220, 4920, 4921], 3) here
5677 : ([5676, 5420, 5419, 5168, 5940, 5941], 3) here
5941 : ([5940, 5678, 5677, 5420, 6210, 6211], 3) here
6488 : ([6769, 6489, 621

### Checking first and last tile on each iteration, by layer.

In [103]:
# we know 1 and 2 both satisfy PD(n) = 3, accounts for layers 0 and 1
overall_count = 2

print(1, ":", 1)
print(2, ":", 2)

a, b = 2, 7
max_prime = -1
layer = 2
while True:
    sl = 6*layer
    a, b = b + 1, b + sl

    # checking formulas
    # First: [6l + 1, 6l - 1, 12l + 5]
    if pbs[sl + 1] + pbs[sl - 1] + pbs[2*sl + 5] == 3:
        overall_count += 1
        max_prime = max(2*sl + 5, max_prime)
        print(a, ":", overall_count)
    
    if overall_count == 2000:
        break

    # checking formulas
    # Last: [6l + 5, 6l - 1, 12l - 7]
    if pbs[sl + 5] + pbs[sl - 1] + pbs[2*sl - 7] == 3:
        overall_count += 1
        max_prime = max(sl + 5, max_prime)
        print(b, ":", overall_count)
    
    if overall_count == 2000:
        break

    layer += 1

print(max_prime)

1 : 1
2 : 2
8 : 3
19 : 4
20 : 5
37 : 6
61 : 7
128 : 8
217 : 9
271 : 10
398 : 11
919 : 12
1519 : 13
1520 : 14
2978 : 15
3170 : 16
4220 : 17
4447 : 18
4681 : 19
5677 : 20
5941 : 21
6488 : 22
8269 : 23
9920 : 24
10621 : 25
12481 : 26
16651 : 27
17558 : 28
22448 : 29
26227 : 30
29701 : 31
34028 : 32
34669 : 33
35317 : 34
35971 : 35
56719 : 36
60920 : 37
61777 : 38
74419 : 39
75367 : 40
80197 : 41
88238 : 42
93458 : 43
110018 : 44
117019 : 45
125461 : 46
136747 : 47
140618 : 48
156637 : 49
169220 : 50
172081 : 51
174968 : 52
182288 : 53
183769 : 54
185257 : 55
214670 : 56
216277 : 57
217891 : 58
246248 : 59
265520 : 60
292970 : 61
302419 : 62
331670 : 63
333667 : 64
362269 : 65
370658 : 66
381278 : 67
407377 : 68
416270 : 69
486019 : 70
498169 : 71
540601 : 72
558578 : 73
590077 : 74
600770 : 75
606151 : 76
672607 : 77
704221 : 78
882920 : 79
1000519 : 80
1053170 : 81
1092637 : 82
1121798 : 83
1181269 : 84
1181270 : 85
1203967 : 86
1215398 : 87
1277269 : 88
1277270 : 89
1281187 : 90
1285111